# OpenVINOを使った物体検出プログラムの基礎を学ぶ
OpenVINOを使うと物体検出(object detection)プログラムも簡単に作ることができます。  
ここではSSD(Single Shot multi-box Detection)モデルを使った物探検出プログラムの基礎を学びます。

### 必要なPythonモジュールをインストールする
このコースでは`matplotlib`を使って画像表示を行うので、モジュールをインストールしておきます。

In [ ]:
# Linux
!pip3 install matplotlib

In [ ]:
# Windows
!pip install matplotlib

### 入力画像とラベルデータの準備
まずは推論に使用する入力画像ファイルと、クラスラベルのテキストファイルをOpenVINOのdemoディレクトリからコピーしてきます。

In [ ]:
# Linux
!cp $INTEL_OPENVINO_DIR/deployment_tools/demo/car_1.bmp .
!cp $INTEL_OPENVINO_DIR/deployment_tools/open_model_zoo/data/dataset_classes/voc_20cl_bkgr.txt .

In [ ]:
# Windows 
!copy "%INTEL_OPENVINO_DIR%\deployment_tools\demo\car_1".bmp .
!copy "%INTEL_OPENVINO_DIR%\deployment_tools\open_model_zoo\data\dataset_classes\voc_20cl_bkgr.txt" .

コピーしてきた推論入力の絵を表示して確認します。  
**Note:** `IPython.display.Image`ではBMPファイルの表示ができないので、ここでは`OpenCV`と`matplotlib`を使用しています。

In [ ]:
%matplotlib inline
import cv2
import matplotlib.pyplot as plt
img=cv2.imread('car_1.bmp')
img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.show()

### 推論に使用するIRモデルデータの準備
推論に使用するモデルを`Model downloader`でダウンロードし、`Model converter`でIRモデルに変換します。  
モデルは軽量な`mobilenet-ssd`モデルを使用します。

In [ ]:
# Linux
!python3 $INTEL_OPENVINO_DIR/deployment_tools/tools/model_downloader/downloader.py --name mobilenet-ssd
!python3 $INTEL_OPENVINO_DIR/deployment_tools/tools/model_downloader/converter.py  --name mobilenet-ssd --precisions FP16
!ls public/mobilenet-ssd/FP16 -l

In [ ]:
# Windows
!python "%INTEL_OPENVINO_DIR%\deployment_tools\tools\model_downloader\downloader.py" --name mobilenet-ssd
!python "%INTEL_OPENVINO_DIR%\deployment_tools\tools\model_downloader\converter.py"  --name mobilenet-ssd --precisions FP16
!dir public\mobilenet-ssd\FP16

----
ここからPythonプログラム本体となります。  

### プログラムで使用するモジュールをインポートする

In [ ]:
import cv2
import numpy as np
from openvino.inference_engine import IECore

### クラスラベルテキストデータを読み込む

In [ ]:
label = open('voc_20cl_bkgr.txt').readlines()

print(len(label), 'labels read')   # 読み込んだラベルの個数を表示
print(label)                       # ラベルを表示

### Inference Engineオブジェクトを生成する
- Inference Engineコアオブジェクトを生成
- モデルデータの読み込み
- 入出力ブロブ(バッファ)の情報取得

In [ ]:
# Inference Engineコアオブジェクトの生成
ie = IECore()

# IRモデルファイルの読み込み
model = './public/mobilenet-ssd/FP16/mobilenet-ssd'
net = ie.read_network(model=model+'.xml', weights=model+'.bin')

# 入出力blobの名前の取得、入力blobのシェイプの取得
input_blob_name  = list(net.input_info.keys())[0]
output_blob_name = list(net.outputs.keys())[0]
batch,channel,height,width = net.input_info[input_blob_name].tensor_desc.dims

# モデルの情報の表示
print(input_blob_name, output_blob_name, batch, channel, height, width)
print(net.outputs[output_blob_name].shape)

### モデルをIE coreオブジェクトにロードする
読み込んだネットワークオブジェクトをInference engineプラグインにセットします。 

In [ ]:
exec_net = ie.load_network(network=net, device_name='CPU', num_requests=1)

### 推論入力データを準備する
推論入力画像を読み込み、入力ブロブのシェイプに合わせて変形します。

In [ ]:
print('input blob: name="{}", N={}, C={}, H={}, W={}'.format(input_blob_name, batch, channel, height, width))
img    = cv2.imread('car_1.bmp')
in_img = cv2.resize(img, (width,height))
in_img = in_img.transpose((2, 0, 1))
in_img = in_img.reshape((1, channel, height, width))

print(in_img.shape)

### 推論を実行する
`infer()` APIはブロッキング関数です。推論が終了すると制御が戻り、次の行を実行します。  
入力は`{入力blob名:入力データ}`の辞書の形で渡します。

**ここまでのコードは画像分類(classification)の時と変わっていません(モデル、ラベル、入力画像ファイル名だけが違う)。OpenVINOでの推論プログラムはモデルに応じた結果の処理部分以外は共通で使える部分が多いのも特徴です**

In [ ]:
res = exec_net.infer(inputs={input_blob_name: in_img})

print(res[output_blob_name].shape)

### 推論結果を表示する  
今回使用しているmobilenet-ssdモデル100のオブジェクト候補を出力し、各オブジェクトごとに7つのデータを出力します(シェイプが[1,1,100,7])。  
各オブジェクト毎のデータは[`id`, `class#`, `confidence`, `xmin`, `ymin`, `xmax`, `ymax`]の７つのパラメータが出てきます。  
class#はクラス番号を、confidenceは「そのクラスらしさ」を0.0-1.0 (1.0=100%)で表します。(`xmin`,`ymin`)-(`xmax`,`ymax`)がオブジェクトを囲む箱(bounding box)の座標を示します。bounding boxの座標も0.0-1.0で出力され、画像の左上が(0.0, 0.0)、画像の右下が(1.0, 1.0)となるので画像のピクセル数を掛けて座標値に変換し、黄色の箱とラベル名を描画しています。  
最後に`matplotlib`を使用して画像を表示させています。

In [ ]:
print('output blob: name="{}", shape={}'.format(output_blob_name, net.outputs[output_blob_name].shape))
result = res[output_blob_name][0][0]
img_h, img_w, _ = img.shape
for obj in result:
    imgid, clsid, confidence, x1, y1, x2, y2 = obj
    if confidence>0.6:
        x1 = int(x1 * img_w)
        y1 = int(y1 * img_h)
        x2 = int(x2 * img_w)
        y2 = int(y2 * img_h)
        cv2.rectangle(img, (x1, y1), (x2, y2), (0,255,255), thickness=4 )
        cv2.putText(img, label[int(clsid)][:-1], (x1, y1), cv2.FONT_HERSHEY_PLAIN, fontScale=4, color=(0,255,255), thickness=4)

%matplotlib inline
import matplotlib.pyplot as plt
img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.show()

----
ここまででOpenVINOを使用した物体検出プログラムの基礎を学びました。  
見てきたように、推論結果処理部分が違うだけで大部分が画像推論(classification)プログラムと同じになっていました。  
OpenVINOではモデルの入力と出力のデータフォーマットさえ分かっていれば、詳細なモデルの動作などを理解していなくてもディープラーニングを使ったプログラムを簡単に作成することが可能です。

## Next => 非同期推論実行方法の基礎 - [classification-async-single.ipynb](./classification-async-single.ipynb)